In [1]:
from IPython.display import display, HTML
import pandas as pd
from pathlib import Path

Grab all anomaly names from the game files. For simplicity, I suggest creating a shortcut that leads to the Stellaris directory inside Steam. Alternatively, replace steam_stellaris with the full path. The full path looks like C:\Program Files (x86)\Steam\steamapps\common\Stellaris.

In [2]:
stellaris_path = Path('steam_stellaris')
localisation_path = stellaris_path.joinpath('localisation', 'english')

This is a bit of a pain because the events are spread across many files and their identification isn't consistent.

In [3]:
possible_identifiers = [
    '_category:0', '_category_temp:0', '_category: ', 
    '_CAT: ', '_CAT:1', '_cat:0', '_CAT:0'
]
event_names = []
for file_path in localisation_path.glob('*.yml'):
    with open(file_path, 'r') as f:
        for line in f.readlines():
            name = ''
            for identifier in possible_identifiers:
                if identifier in line:
                    name = line.split(identifier)[1].strip().strip('"')
            if name and not '$' in name:
                event_names.append(name)
event_names = pd.DataFrame({'anomaly': event_names})

Grab the anomaly tables from the wiki

In [4]:
anomaly_url = 'https://stellaris.paradoxwikis.com/Anomaly'
tables = pd.read_html(anomaly_url)
f'Found {len(tables)} tables'

'Found 20 tables'

Unfortunately, not all table defintions are actually useful. Inspect tables to find which and modify below if needed.

In [5]:
useful_table_indices = [0, 2, 4]

Merge the tables under common column names.

In [6]:
dfs = []
for i in useful_table_indices:
    table = tables[i]
    description = 'Reward' if 'Reward' in table.columns else 'Possible outcomes'
    dfs.append(
        pd.DataFrame({
            'anomaly': table['Anomaly'],  
            'description': table[description] + ' ' + table['Celestial body']
        })
    )
df = pd.concat(dfs)

Handle special cases.

In [7]:
df.loc[df['anomaly'] == 'Ice Giant', 'anomaly'] = 'Ice Ice Giant'

df.loc[df['anomaly'].str.contains('Debris Field'), 'anomaly'] = 'Debris Field'
new_desc = df[df['anomaly'] == 'Debris Field'].groupby(['anomaly']).agg(' OR '.join)
df.loc[df['anomaly'] == 'Debris Field', 'description'] = new_desc['description'].values[0]
df = df.drop_duplicates(subset=['anomaly', 'description'])

In [8]:
merged = event_names.merge(
    df, 
    left_on=event_names['anomaly'].str.lower(), 
    right_on=df['anomaly'].str.lower(),
    how='outer'
)
display(HTML(
    merged[merged['anomaly_y'].isna()].to_html()
))


,key_0,anomaly_x,anomaly_y,description
3,a lush planet,A Lush Planet,NaN,NaN
4,a planetary machine,A Planetary Machine,NaN,NaN
6,abandoned observation post,Abandoned Observation Post,NaN,NaN
9,abnormal conditions,Abnormal Conditions,NaN,NaN
15,alien site,Alien Site,NaN,NaN
21,ancient manufactory,Ancient Manufactory,NaN,NaN
23,ancient signs of life,Ancient Signs of Life,NaN,NaN
37,backgrounds,Backgrounds,NaN,NaN
38,between land and sea,Between Land and Sea,NaN,NaN
39,billowing sands,Billowing Sands,NaN,NaN


Pair key words in the anomaly description. Order matters such that the first category to be found is the chosen category.

In [9]:
categories = dict(
    red=['scientist dies', 'Paranoid trait'],
    green=[
        'Any Habitable Planet', 'technology', 'minerals',
        'influence', 'unity', 'corvettes'
    ],
    yellow=[
        'scaled', 'deposit', 'archaeological', 'L-Gate',
        'Physics Research', 'Society Research', 'Engineering Research'],
)

In [10]:
def like_function(x):
    group = 'white'
    for key, values in categories.items():
        for value in values:
            if value.lower() in x.lower():
                return key
    return group

df['category'] = df['description'].apply(like_function)

In [11]:
df.shape

(165, 3)

In [12]:
df['category'].value_counts()

category
green     82
yellow    61
white     19
red        3
Name: count, dtype: int64

In [13]:
display(HTML(
    df[df['category'] == 'white'].to_html()
))

,anomaly,description,category
20,Doppler Effect,The Ransomeers event chain Uninhabitable Planet,white
37,Rapid Desertification,Planet gains the Alien Infestation modifier and a special project to replace it with Lush and the Bountiful Plains and Green Hills planetary features Can choose to instead gain +10% Society from Jobs and −5% Housing for 10 years if not Xenophobe Can choose to instead gain 500 Food if not Pacifist or Xenophile Can choose to instead gain a pre-sapient pop with the Delicious and Unintelligent trait on every planet if Xenophobe Dry Planet,white
38,Spotty Greenery,Hyperfertile Valley planetary feature Special project to add the Ancient Mining Site planetary feature and give the science ship scientist the Expertise: Industry trait Wet Planet,white
41,Arid Wastes,Planet gains +50% Energy from Jobs for 2 years after being colonized and 2 Frozen Gas Lake planetary features Arid,white
45,"An Asteroid, Carved",Shrines to the Old Gods event chain Asteroid,white
46,An Invitation,Option to spend 200 Energy to gain a random reward Asteroid,white
62,Orbital Speed Demon,Orbital Speed Demon event chain Asteroid,white
70,On the Barren Plains,Limbo event chain Barren,white
83,Minesweeper,Science ship scientist gains the Expertise: New Worlds trait Gaia,white
89,Clouds Dance,The Gas Giant is terraformed into a Barren World and the science ship scientist gains 200 experience Gas Giant,white
